In [ ]:
import numpy as np
import plotly.graph_objects as go

In [ ]:
faiss_f32 = np.load("stats/FAISS-HNSW-f32-1536d.npz")
faiss_f16 = faiss_f32 # np.load("stats/FAISS-HNSW-f16-1536d.npz")
faiss_i8 = faiss_f32 # np.load("stats/FAISS-HNSW-i8-1536d.npz")
usearch_f32 = np.load("stats/USearch-HNSW-f32-1536d.npz")
usearch_f16 = np.load("stats/USearch-HNSW-f16-1536d.npz")
usearch_i8 = np.load("stats/USearch-HNSW-i8-1536d.npz")

Every set of benchmarks contains an array called `construction_time` and `search_time`. Each of them contains stats for 1000 benchmark steps constructing and evaluating the performance of 1536-dimensional index of 100 Million entries. So those arrays contain 1000 durations for time intervals in seconds.

In [ ]:
for fraction in range(10):
    parts_per_fraction = len(faiss_f32['search_time']) // 10
    start = parts_per_fraction * fraction
    end = start + parts_per_fraction

    # Compute the sum construction and search time for each dataset
    faiss_f32_search_sum = 1 / np.sum(faiss_f32['search_time'][start:end])
    faiss_f16_search_sum = 1 / np.sum(faiss_f16['search_time'][start:end])
    faiss_i8_search_sum = 1 / np.sum(faiss_i8['search_time'][start:end])
    usearch_f32_search_sum = 1 / np.sum(usearch_f32['search_time'][start:end])
    usearch_f16_search_sum = 1 / np.sum(usearch_f16['search_time'][start:end])
    usearch_i8_search_sum = 1 / np.sum(usearch_i8['search_time'][start:end])

    # Create a horizontal bar chart using Plotly

    faiss_sums = [faiss_f32_search_sum, faiss_f16_search_sum, faiss_i8_search_sum]
    usearch_sums = [usearch_f32_search_sum, usearch_f16_search_sum, usearch_i8_search_sum]
    labels = ['32-bit float', '16-bit float', '8-bit int']

    fig = go.Figure()

    # Add bars for FAISS
    fig.add_trace(go.Bar(
        y=labels,
        x=faiss_sums,
        name='FAISS',
        orientation='h'
    ))

    # Add bars for USearch
    fig.add_trace(go.Bar(
        y=labels,
        x=usearch_sums,
        name='USearch',
        orientation='h'
    ))

    # Add annotations for USearch bars
    for i, label in enumerate(labels):
        ratio = usearch_sums[i] / faiss_sums[i]
        fig.add_annotation(dict(
            x=usearch_sums[i] / 2, 
            y=i + 0.18,  # Using the index for precise placement
            text=f'USearch: {ratio:.2f}x higher throughput',
            font=dict(color="white"),
            showarrow=False,
            xanchor='center',
            yanchor='middle',

        ))

        # Add "FAISS" next to FAISS bars
        fig.add_annotation(dict(
            x=faiss_sums[i] + (0.15 * faiss_sums[i]),  # Some offset for visual clarity
            y=i - 0.18,  # Slight offset to move the annotation up a bit
            text="FAISS",
            font=dict(color="black"),
            showarrow=False,
            xanchor='left',
            yanchor='middle',
        ))

        # Add annotations for bar labels on top of each group
        fig.add_annotation(dict(
            x=0,  # Placing label in the center of the group
            y=i + 0.50,  # Slightly above the bars; adjust this for desired position
            text=label,
            font=dict(color="black", size=14, family="Arial, bold"),
            showarrow=False,
            xanchor='left',
            yanchor='middle',
        ))

    # Compute the speed comparison ratio
    speed_comparison_ratio = usearch_i8_search_sum / faiss_f32_search_sum

    # Customize layout
    fig.update_layout(
        title=dict(
            text=f'Search Throughput of USearch vs FAISS on <b>{(fraction+1)*10} Million</b> 1536d Vectors <br> <b>{speed_comparison_ratio:.2f}x</b> performance gap across engines and vector types',
            x=0.5, # Center the title
            xanchor='center'
        ),
        barmode='group',
        showlegend=False,
        plot_bgcolor='white',
        paper_bgcolor='white',
        height=600,  
        width=800,   
        bargap=0.3,  # Increases the gap between bars within a group (e.g., 0.15 for a 15% gap)
        bargroupgap=0.2,  # Increases the gap between groups (e.g., 0.3 for a 30% gap)
        xaxis=dict(
            showline=False,
            showgrid=False,
            zeroline=False,
            showticklabels=False
        ),
        yaxis=dict(
            showticklabels=False  # Hiding the original y-axis labels
        )
    )

    fig.show()


In [ ]:
np.sum(np.load("stats/FAISS-HNSW-f32-1536d.npz")["construction_time"][:300])

In [ ]:
np.sum(np.load("stats/USearch-HNSW-f32-1536d.npz")["construction_time"][:300])

In [ ]:
np.sum(np.load("stats/FAISS-HNSW-f32-1536d.npz")["search_time"][:400])

In [ ]:
np.sum(np.load("stats/USearch-HNSW-f32-1536d.npz")["search_time"][:400])

In [ ]:
len(np.load("stats/FAISS-HNSW-f32-1536d.npz")["search_time"])